<a href="https://colab.research.google.com/github/brahma2024/gpt_code/blob/dev/gpt_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a GPT
Companion notebook to the GPT notebook.

In [ ]:
import base64
import requests

In [ ]:
# We start with dataset to train on
!wget https://raw.githubusercontent.com/brahma2024/gpt_code/master/input.txt

--2024-04-08 00:40:45--  https://raw.githubusercontent.com/brahma2024/gpt_code/master/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10008 (9.8K) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   9.77K  --.-KB/s    in 0s      

2024-04-08 00:40:45 (55.2 MB/s) - ‘input.txt’ saved [10008/10008]



In [ ]:
# read the input text
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  10008


In [ ]:
# lets look at the first 1000 characters
print(text[:1000])

Naman:
The top in a world by susphoring grace.

LUCIO:
We muse hath resistes him so sovere: son't his other wrough
stands of coverent sh'd: he has here, and stand it
and poor exceeder or a Henry's last, stay
not in faith, forewell's base of graves, thanks, happy comparel,
warmentfully: may as face by the courst, that strangth
errise hath breathed. Hastings come to Valenting.

HERMIONE:
Well have been bolly poor late
Is the lords.

ABELLA:
Let's found: I will kind him;
I do braw'sy him business wherein far his face.

LUCENTIO:
He is last afford: make him diseably to London,
Take him great Hastings, boldness in his natic keeps,
To oftragn lost me ready glust through the house.
Why chose that I dares it be a Montague.

MONTAGUE:
Woe's Claudly Haste of his own at last the Volscient,
And seen'd helpit: bearn to do it be, and most hop,
Miscause's more conterar than without this lambs
Shall down appla fortune flight flowers.

FRIAR LAUAURENCE:
His son, do your morself, that leaven your honour

In [ ]:
# here are all unique characters that occur in this set
chars = sorted(list(set(text)))
vocab_size = len(chars)
print('number of characters: ', vocab_size)

number of characters:  59


In [ ]:
# creating a mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars) }
#for i,ch in enumerate(chars):
  #print(f"{ch}:{i}")
print("stoi = ", stoi)

stoi =  {'\n': 0, ' ': 1, '!': 2, "'": 3, ',': 4, '-': 5, '.': 6, ':': 7, ';': 8, '?': 9, 'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15, 'G': 16, 'H': 17, 'I': 18, 'K': 19, 'L': 20, 'M': 21, 'N': 22, 'O': 23, 'P': 24, 'Q': 25, 'R': 26, 'S': 27, 'T': 28, 'U': 29, 'V': 30, 'W': 31, 'Y': 32, 'a': 33, 'b': 34, 'c': 35, 'd': 36, 'e': 37, 'f': 38, 'g': 39, 'h': 40, 'i': 41, 'j': 42, 'k': 43, 'l': 44, 'm': 45, 'n': 46, 'o': 47, 'p': 48, 'q': 49, 'r': 50, 's': 51, 't': 52, 'u': 53, 'v': 54, 'w': 55, 'x': 56, 'y': 57, 'z': 58}


In [ ]:
itos = {i:ch for i,ch in enumerate(chars)}
#for i,ch in enumerate(chars):
#  print(f"{i}:{ch}")
print("itos = ", itos)

itos =  {0: '\n', 1: ' ', 2: '!', 3: "'", 4: ',', 5: '-', 6: '.', 7: ':', 8: ';', 9: '?', 10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G', 17: 'H', 18: 'I', 19: 'K', 20: 'L', 21: 'M', 22: 'N', 23: 'O', 24: 'P', 25: 'Q', 26: 'R', 27: 'S', 28: 'T', 29: 'U', 30: 'V', 31: 'W', 32: 'Y', 33: 'a', 34: 'b', 35: 'c', 36: 'd', 37: 'e', 38: 'f', 39: 'g', 40: 'h', 41: 'i', 42: 'j', 43: 'k', 44: 'l', 45: 'm', 46: 'n', 47: 'o', 48: 'p', 49: 'q', 50: 'r', 51: 's', 52: 't', 53: 'u', 54: 'v', 55: 'w', 56: 'x', 57: 'y', 58: 'z'}


In [ ]:
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integeres
decode = lambda l: ''.join([itos[i] for i in l]) # decode: take a list of integers, output a string

print(encode("hi there"))
print(decode(encode("hi there")))

[40, 41, 1, 52, 40, 37, 50, 37]
hi there


In [ ]:
# Lets now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earlier will to GPT look like this

torch.Size([10008]) torch.int64
tensor([22, 33, 45, 33, 46,  7,  0, 28, 40, 37,  1, 52, 47, 48,  1, 41, 46,  1,
        33,  1, 55, 47, 50, 44, 36,  1, 34, 57,  1, 51, 53, 51, 48, 40, 47, 50,
        41, 46, 39,  1, 39, 50, 33, 35, 37,  6,  0,  0, 20, 29, 12, 18, 23,  7,
         0, 31, 37,  1, 45, 53, 51, 37,  1, 40, 33, 52, 40,  1, 50, 37, 51, 41,
        51, 52, 37, 51,  1, 40, 41, 45,  1, 51, 47,  1, 51, 47, 54, 37, 50, 37,
         7,  1, 51, 47, 46,  3, 52,  1, 40, 41, 51,  1, 47, 52, 40, 37, 50,  1,
        55, 50, 47, 53, 39, 40,  0, 51, 52, 33, 46, 36, 51,  1, 47, 38,  1, 35,
        47, 54, 37, 50, 37, 46, 52,  1, 51, 40,  3, 36,  7,  1, 40, 37,  1, 40,
        33, 51,  1, 40, 37, 50, 37,  4,  1, 33, 46, 36,  1, 51, 52, 33, 46, 36,
         1, 41, 52,  0, 33, 46, 36,  1, 48, 47, 47, 50,  1, 37, 56, 35, 37, 37,
        36, 37, 50,  1, 47, 50,  1, 33,  1, 17, 37, 46, 50, 57,  3, 51,  1, 44,
        33, 51, 52,  4,  1, 51, 52, 33, 57,  0, 46, 47, 52,  1, 41, 46,  1, 38,
        

In [ ]:
# Lets now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([22, 33, 45, 33, 46,  7,  0, 28, 40])

In [ ]:
# Model Training logic
# where x holds the input values and y holds the output values. y here is x-offset-by-1
# Essentially we are trying to train the model to identify the pattern to identify the next token given a specific sequence of token
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(f"x: {x} '\ny: {y}")
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

x: tensor([22, 33, 45, 33, 46,  7,  0, 28]) '
y: tensor([33, 45, 33, 46,  7,  0, 28, 40])
when input is tensor([22]) the target: 33
when input is tensor([22, 33]) the target: 45
when input is tensor([22, 33, 45]) the target: 33
when input is tensor([22, 33, 45, 33]) the target: 46
when input is tensor([22, 33, 45, 33, 46]) the target: 7
when input is tensor([22, 33, 45, 33, 46,  7]) the target: 0
when input is tensor([22, 33, 45, 33, 46,  7,  0]) the target: 28
when input is tensor([22, 33, 45, 33, 46,  7,  0, 28]) the target: 40


In [ ]:
# training the model
torch.manual_seed(2207) # since we are going to sample random chunks of data from the training data set, we are setting a seed for randomization
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # what is the maximum context length of predictions?

def get_batch(split):
  # generate a small bacth of data of inputs x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  print(f'ix : {ix}')
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[1+i:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print(f"inputs: '\n {xb} '\n {xb.shape}")
print(f"targets: '\n {yb} '\n {yb.shape}")

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")

ix : tensor([4546, 1373, 3594, 1762])
inputs: '
 tensor([[ 0, 13, 29, 12, 17, 14, 27, 27],
        [50, 38, 53, 44,  1, 55, 33, 51],
        [54, 37,  1, 45, 57,  1, 35, 44],
        [55, 41, 38, 37,  4,  1, 33, 51]]) '
 torch.Size([4, 8])
targets: '
 tensor([[13, 29, 12, 17, 14, 27, 27,  1],
        [38, 53, 44,  1, 55, 33, 51,  1],
        [37,  1, 45, 57,  1, 35, 44, 47],
        [41, 38, 37,  4,  1, 33, 51,  1]]) '
 torch.Size([4, 8])
when input is [0] the target: 13
when input is [0, 13] the target: 29
when input is [0, 13, 29] the target: 12
when input is [0, 13, 29, 12] the target: 17
when input is [0, 13, 29, 12, 17] the target: 14
when input is [0, 13, 29, 12, 17, 14] the target: 27
when input is [0, 13, 29, 12, 17, 14, 27] the target: 27
when input is [0, 13, 29, 12, 17, 14, 27, 27] the target: 1
when input is [50] the target: 38
when input is [50, 38] the target: 53
when input is [50, 38, 53] the target: 44
when input is [50, 38, 53, 44] the target: 1
when input is [50, 38, 

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(2207)

class BigramLanguageModel(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from the lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets):

    #idx and targets